In [ ]:
using Pkg
Pkg.activate("..")
using JPEC
using Plots

## Overview
In this notebook, I attempt to show where I am seeing bugs in the equilibrium knots/splines when running the Solovev ideal example. I noticed in the fourfit functions, I was seeing discrepancies for the metric tensor computation, and traced them down to the following function calls giving sufficiently different values from the same values when I dumped the same values from the Fortran code run with the same input files (above what is to be expected due to machine precision/floating point errors). I have put the Fortran data from this dump in the soloviev_ideal example, but could also be obtained by adding the relevant write statements for `fourfit_make_metric` and `fourfit_make_matrix` within `fourfit.F`. 

I **think** this is an issue with equil and not splines, since I have verified that if I use the Fortran knot values, I get the correct matrices, which involves a creating a Fourier spline for the metric tensor, but I am not 100% sure.

The relevant data for these calculations is:

fortran_rzphi_knots.dat: rzphi%f(1:4), fx(1:4), fy(1:4) at each psi and theta

fortran_sq_knots.dat: sq%fs1(2), sq%fs(4), sq%fs1(4), -sq%fs1(1) (p1, q, q1, and jtheta, respectively) at each psi

In [ ]:
### Set up the equilibrium data for the solovev ideal example
equil = Equilibrium.setup_equilibrium("soloviev_ideal/equil.toml")

## 1D Profiles

First, I will show the errors I was finding with the 1D profiles from sq

In [ ]:
# Extract knots from the sq data that we will compare to the Fortran values
psi = equil.sq.xs
p1 = equil.sq.fs1[:, 2]
q = equil.sq.fs[:, 4]
q1 = equil.sq.fs1[:, 4]
jtheta = -equil.sq.fs1[:, 1]

# Read the Fortran data into equivalent arrays
lines = readlines("soloviev_ideal/fortran_sq_knots.dat")
data = [parse.(Float64, split(l)) for l in lines[2:end]]
ipsi_f = zeros(size(psi))
psi_f = zeros(size(psi))
p1_f = zeros(size(psi))
q_f = zeros(size(psi))
q1_f = zeros(size(psi))
jtheta_f = zeros(size(psi))
for row in data
    ipsi_f[Int(row[1])+1] = row[1]  # shift to Julia indexing
    psi_f[Int(row[1])+1] = row[2]
    p1_f[Int(row[1])+1] = row[3]
    q_f[Int(row[1])+1] = row[4]
    q1_f[Int(row[1])+1] = row[5]
    jtheta_f[Int(row[1])+1] = row[6]
end

# Plot q profile comparison (log scale)
plot(psi, abs.((q - q_f) ./ q_f), label="rel error", xlabel="psi", ylabel="|q(Julia) - q(Fortran)| / |q(Fortran)|", title="q profile comparison", yscale=:log10)
display(current())

# Plot p1 profile comparison (log scale)
plot(psi, abs.((p1 - p1_f) ./ p1_f), label="rel error", xlabel="psi", ylabel="|p1(Julia) - p1(Fortran)| / |p1(Fortran)|", title="p1 profile comparison", yscale=:log10)
display(current())

# Plot q1 profile comparison (log scale)
plot(psi, abs.((q1 - q1_f) ./ q1_f), label="rel error", xlabel="psi", ylabel="|q1(Julia) - q1(Fortran)| / |q1(Fortran)|", title="q1 profile comparison", yscale=:log10)
display(current())

# Plot jtheta profile comparison (log scale)
plot(psi, abs.((jtheta - jtheta_f) ./ jtheta_f), label="rel error", xlabel="psi", ylabel="|jtheta(Julia) - jtheta(Fortran)| / |jtheta(Fortran)|", title="jtheta profile comparison", yscale=:log10)
display(current())

We can see that the q and p1 profiles look good in terms of error, but the q1 profile has a large spike at the magnetic axis and jtheta has large relative errors throughout. These values are needed for assembling the matrices in fourfit_make_matrix, and this was propagating to cause large errors in those values as well

## 2D Profiles

Now, we will look at the relative and absolute errors resulting from the rzphi values used to assemble the metric tensor in fourfit_make_metric

In [ ]:
# Assemble arrays of Julia equilibrium values. Note that there is a bicube eval here to match the line in Fortran, but the
# evaluation occurs on the knot values so I don't think this would be a spline issue, but again not 100% sure.
rzphi = equil.rzphi
psi = rzphi.xs
theta = rzphi.ys
mpsi = length(rzphi.xs) - 1
mtheta = length(rzphi.ys) - 1
f_store  = Array{Float64}(undef, mpsi + 1, mtheta + 1, 4) 
fx_store = Array{Float64}(undef, mpsi + 1, mtheta + 1, 4)
fy_store = Array{Float64}(undef, mpsi + 1, mtheta + 1, 4)
f_store_f  = Array{Float64}(undef, mpsi + 1, mtheta + 1, 4) 
fx_store_f = Array{Float64}(undef, mpsi + 1, mtheta + 1, 4)
fy_store_f = Array{Float64}(undef, mpsi + 1, mtheta + 1, 4)
for ipsi in 1:(mpsi+1)
    for jtheta in 1:(mtheta+1)
        f, fx, fy = SplinesMod.bicube_eval(rzphi, rzphi.xs[ipsi], rzphi.ys[jtheta], 1)
        f_store[ipsi, jtheta, :]  .= f
        fx_store[ipsi, jtheta, :] .= fx
        fy_store[ipsi, jtheta, :] .= fy
    end
end

# Read in equivalent Fortran data
lines = readlines("soloviev_ideal/fortran_rzphi_knots.dat")
data = [parse.(Float64, split(l)) for l in lines[2:end]]
for row in data
    ipsi   = Int(row[1]) + 1   # shift to Julia indexing
    itheta = Int(row[2]) + 1
    f_store_f[ipsi, itheta, :]  .= row[3:6]
    fx_store_f[ipsi, itheta, :] .= row[7:10]
    fy_store_f[ipsi, itheta, :] .= row[11:14]
end

# Plot relative error between Julia and Fortran values
labels = ["(1)", "(2)", "(3)", "(4)"]
for i in 1:4
    # Relative error for f
    rel_err_f = abs.((f_store[:, :, i] .- f_store_f[:, :, i]) ./ f_store_f[:, :, i])
    contour(
        psi,
        theta,
        rel_err_f',
        title="Relative Error: f$(labels[i])",
        xlabel="psi",
        ylabel="theta",
        colorbar_title="rel error",
        colorbar_scale=:log10
    )
    display(current())

    # Relative error for fx
    rel_err_fx = abs.((fx_store[:, :, i] .- fx_store_f[:, :, i]) ./ fx_store_f[:, :, i])
    contour(
        psi,
        theta,
        rel_err_fx',
        title="Relative Error: fx$(labels[i])",
        xlabel="psi",
        ylabel="theta",
        colorbar_title="rel error",
        colorbar_scale=:log10
    )
    display(current())

    # Relative error for fy
    rel_err_fy = abs.((fy_store[:, :, i] .- fy_store_f[:, :, i]) ./ fy_store_f[:, :, i])
    contour(
        psi,
        theta,
        rel_err_fy',
        title="Relative Error: fy$(labels[i])",
        xlabel="psi",
        ylabel="theta",
        colorbar_title="rel error",
        colorbar_scale=:log10
    )
    display(current())
end

In [ ]:
labels = ["(1)", "(2)", "(3)", "(4)"]
for i in 1:4
    # Absolute error for f
    abs_err_f = abs.(f_store[:, :, i] .- f_store_f[:, :, i])
    contour(
        psi,
        theta,
        abs_err_f',
        title="Absolute Error: f$(labels[i])",
        xlabel="psi",
        ylabel="theta",
        colorbar_title="abs error",
        colorbar_scale=:log10
    )
    display(current())

    # Absolute error for fx
    abs_err_fx = abs.(fx_store[:, :, i] .- fx_store_f[:, :, i])
    contour(
        psi,
        theta,
        abs_err_fx',
        title="Absolute Error: fx$(labels[i])",
        xlabel="psi",
        ylabel="theta",
        colorbar_title="abs error",
        colorbar_scale=:log10

    )
    display(current())

    # Absolute error for fy
    abs_err_fy = abs.(fy_store[:, :, i] .- fy_store_f[:, :, i])
    contour(
        psi,
        theta,
        abs_err_fy',
        title="Absolute Error: fy$(labels[i])",
        xlabel="psi",
        ylabel="theta",
        colorbar_title="abs error",
        colorbar_scale=:log10
    )
    display(current())
end

It is a bit difficult to see on the relative error plots since there are a few extremely large spikes (that I think occur when the value is below machine precision and the relative error is inaccurate), but I think the jist here is to look at the absolute error in fx(2:4), where there is an anomalous spike near the axis. This difference was drectly affecting the calculations. I **think** the rest of the values are ok, but we could double check by filtering out errors of values below 10^-10 or something and verifying that all relative errors for values above this limit are small

## Summary

Based on these plots, I think the mostly likely source of error is how the first grid point is dealt with between the two codes. Most likely the actual spline evals are ok, and it seems like for most of the rest of the knots (besides jtheta perhaps?) the errors are small